<a href="https://colab.research.google.com/github/redish0317/cp1/blob/main/cp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def crawl_id():
    url = 'http://www.wanted.jobs/api/v4/jobs?'

    params ={1656232918453:'',    #사용자번호?
    'country': 'all',
    'tag_type_ids': 873,    #직무 카테고리 고유 id
    'job_sort': 'job.latest_order',    #최신순 정렬
    'locations': all,
    'years': -1,    #경력 이상
    'years': -1,    #경력 이하    경력상관없이 검색하려면 -1
    'limit': 10,    #한 번에 조회 가능한 수 (최대100)
    'offset': 10}    #조회할 게시물의 첫 index        ex) limit=100 offset=10  => 10번게시물부터 110번게시물까지 크롤링

    r = requests.get(url,
                     params = params)
    r = r.json()
    df = pd.json_normalize(r['data'])
    return df


def crawl_job(id_list):
    df = pd.DataFrame()
    for id in id_list:
        url = f'https://www.wanted.jobs/api/v4/jobs/{id}?1656259528432'
        r = requests.get(url)
        r = r.json()['job']
        df = pd.concat([df,pd.json_normalize(r)],axis=0)
    return df.reset_index(drop=True)


def engineering(df):
    drop_col = ['status','is_like','is_bookmark','compare_country','matching_score','reward.formatted_total','reward.formatted_recommender','reward.formatted_recommendee','title_img.origin','title_img.thumb',
            'logo_img.origin','logo_img.thumb']
    df = df.drop(drop_col,axis=1)
    return df

In [4]:
df_id = crawl_id()

df_job = crawl_job(df_id['id'])

df = pd.concat([df_id,df_job],axis=1)

#df = engineering(df)

df.head()

,status,is_like,is_bookmark,compare_country,due_time,like_count,id,matching_score,position,score,...,logo_img.thumb,title_img.origin,title_img.thumb,reward.formatted_total,reward.formatted_recommender,reward.formatted_recommendee,address.geo_location.bounds.northeast.lat,address.geo_location.bounds.northeast.lng,address.geo_location.bounds.southwest.lat,address.geo_location.bounds.southwest.lng
0,active,False,False,True,None,0,108602,None,JAVA 시니어 백엔드개발자,None,...,https://static.wanted.co.kr/images/wdes/0_5.13...,https://static.wanted.co.kr/images/company/309...,https://static.wanted.co.kr/images/company/309...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN
1,active,False,False,True,None,91,52187,None,프론트엔드 개발 (React),None,...,https://static.wanted.co.kr/images/wdes/0_5.e6...,https://static.wanted.co.kr/images/company/225...,https://static.wanted.co.kr/images/company/225...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN
2,active,False,False,True,None,44,42610,None,React/Next.js 프론트엔드 개발자 (2년 이상),None,...,https://static.wanted.co.kr/images/wdes/0_5.ee...,https://static.wanted.co.kr/images/company/122...,https://static.wanted.co.kr/images/company/122...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN
3,active,False,False,True,2022-07-10,0,112803,None,Backend Engineer - RCS Messaging Solution (Sen...,None,...,https://static.wanted.co.kr/images/wdes/0_5.9f...,https://static.wanted.co.kr/images/company/185...,https://static.wanted.co.kr/images/company/185...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN
4,active,False,False,True,2022-07-10,0,112802,None,Backend Engineer - RCS Messaging Solution (Jun...,None,...,https://static.wanted.co.kr/images/wdes/0_5.9f...,https://static.wanted.co.kr/images/company/185...,https://static.wanted.co.kr/images/company/185...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN


In [6]:
df_id.columns

Index(['status', 'is_like', 'is_bookmark', 'compare_country', 'due_time',
       'like_count', 'id', 'matching_score', 'position', 'score',
       'category_tags', 'reward.formatted_total',
       'reward.formatted_recommender', 'reward.formatted_recommendee',
       'company.id', 'company.industry_name',
       'company.application_response_stats.avg_rate',
       'company.application_response_stats.level',
       'company.application_response_stats.delayed_count',
       'company.application_response_stats.avg_day',
       'company.application_response_stats.remained_count',
       'company.application_response_stats.type', 'company.name',
       'title_img.origin', 'title_img.thumb', 'logo_img.origin',
       'logo_img.thumb', 'address.country', 'address.location'],
      dtype='object')

In [7]:
df_job.columns

Index(['is_crossboarder', 'like_count', 'id', 'is_like', 'due_time', 'score',
       'company_images', 'skill_tags', 'status', 'is_bookmark',
       'is_company_follow', 'compare_country', 'company_tags',
       'matching_score', 'short_link', 'position', 'category_tags',
       'address.country', 'address.full_location',
       'address.geo_location.n_location.lat',
       'address.geo_location.n_location.lng',
       'address.geo_location.n_location.address',
       'address.geo_location.location.lat',
       'address.geo_location.location.lng',
       'address.geo_location.location_type',
       'address.geo_location.viewport.northeast.lat',
       'address.geo_location.viewport.northeast.lng',
       'address.geo_location.viewport.southwest.lat',
       'address.geo_location.viewport.southwest.lng',
       'address.geo_location.bounds', 'address.location',
       'address.country_code', 'detail.requirements', 'detail.main_tasks',
       'detail.intro', 'detail.benefits', 'detail.pr

In [ ]:
df.to_excel('wanted.xlsx')

#폐기

In [ ]:
def make_soup(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup
url = 'https://www.wanted.jobs/wd/109904'

soup = str(make_soup(url))

url_list = ['https://www.wanted.jobs/wd/'+ str(i) for i in df['id']]

#서론
Introduction = soup[soup.find('"jd":"') + 6 : soup.find('\\n\\n\\n주요업무\\n')]

#주요업무, 책임
responsibilities = soup[soup.find('\\n\\n\\n주요업무\\n') + 12 : soup.find('\\n\\n자격요건\\n')]

#요구자격
basic_qualifications = soup[soup.find('\\n\\n자격요건\\n') + 12 : soup.find('\\n\\n우대사항\\n')]

#우대자격
preferred_qualifications = soup[soup.find('\\n\\n우대사항\\n') + 10 : soup.find('\\n\\n혜택 및 복지\\n###')]

#복지
company_benefits = soup[soup.find('\\n\\n혜택 및 복지\\n###') + 16 : soup.find('","company_name":"')]

pd.DataFrame({'Introduction':Introduction,
              'responsibilities':responsibilities,
              'basic_qualifications':basic_qualifications,
              'preferred_qualifications':preferred_qualifications,
              'company_benefits':company_benefits},index=[0])

,Introduction,responsibilities,basic_qualifications,preferred_qualifications,company_benefits
0,"빌리오(BIllyo)\n#크리에이터의 고민 해결!\n콘텐츠 제작 커뮤니티, ‘빌리오...",[주요 업무]\n- 빌리오(Billyo) 모바일 서비스를 위한 API 개발 및 유지...,Python 기반 Restful API 설계 및 구현 경험이 3년 이상 있으시거나 ...,- O2O 플랫폼 비즈니스에 대한 흥미와 관심이 있는 분\n- 크리에이터 시장에 관...,"이런 게 좋아요!\n빌리오 팀원들은 함께하는 동료를 소중히 여기며, 함께 성장하면..."
